In [ ]:
import numpy as np 
import pandas as pd 
import re

import numpy as np 
import requests
from PIL import Image
from io import BytesIO 

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

from collections import Counter
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(rc={'figure.figsize':(12,8)})

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv('/kaggle/input/schopenhauer-work-corpus/Schopenhauer_works_corpus.csv')
df

In [ ]:
# Isolate the book wanted: Beyond Good and Evil
will = df[df['book_title']=='The World As Will And Idea (Vol. 1 of 3)']['text_clean'][8]
tokens = word_tokenize(will)
freq = Counter(tokens)
sorted_freq = dict(sorted(freq.items(), key=lambda x: x[1], reverse=True))
top_25_words = list(sorted_freq.keys())[:25]
top_25_freq = list(sorted_freq.values())[:25]
sns.barplot(y=top_25_words, x=top_25_freq)

# Wordcloud

In [ ]:
def plot_cloud(wordcloud):
    # Set figure size
    plt.figure(figsize=(12, 8))
    # Display image
    plt.imshow(wordcloud) 
    # No axis details
    plt.axis("off");
    

wordcloud = WordCloud(width = 3000, height = 2000, random_state=1, 
                      background_color='black', colormap='Set2', 
                      collocations=False, stopwords = STOPWORDS)
wordcloud.generate_from_frequencies(sorted_freq)
plot_cloud(wordcloud)

# Text Summarization

In [ ]:
will = df[df['book_title']=='The World As Will And Idea (Vol. 1 of 3)']['text'][8]
will = will.encode(encoding="ascii", errors="ignore").decode()
will = re.sub("[\r\n]", " ", will)

In [ ]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') # one time execution
import re

In [ ]:
from nltk.tokenize import sent_tokenize
sentences = []
text = will.split(". ")
for s in text:
    sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x] # flatten list

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

In [ ]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [ ]:
len(word_embeddings)

In [ ]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [ ]:
# function to remove stopwords
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new
# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [ ]:
sentence_vectors = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)

In [ ]:
# from sklearn.metrics.pairwise import cosine_similarity
# sim_mat = np.zeros([len(sentences), len(sentences)])
# for i in range(len(sentences)):
#     for j in range(len(sentences)):
#         if i != j:
#             sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), 
#                                               sentence_vectors[j].reshape(1,100))[0,0]

In [ ]:
# import networkx as nx

# nx_graph = nx.from_numpy_array(sim_mat)
# scores = nx.pagerank(nx_graph)